# Analyzing Portfolio Risk and Return

In [21]:
# Initial imports
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import numpy as np

%matplotlib inline



### Import the Data


In [3]:
# Load .env enviroment variables
load_dotenv()

True

In [4]:
alpaca_api_key = os.getenv("APCA_API_KEY_ID")
alpaca_secret_key = os.getenv("APCA_SECRET_KEY")

In [5]:
# Set current amount of shares
#Tech
my_appl = 50
my_amzn = 50
my_nflx = 50
my_spy = 50
my_goog = 50
my_fab = 50

# ETFs
my_voo = 50
my_vgt = 50

In [6]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [19]:
# Format current date as ISO format
start_date = pd.Timestamp("2020-08-07", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-09-07", tz="America/New_York").isoformat()
# Set the tickers
tickers = ["FB", "APPL","AMZN","NFLX","GOOG","SPY"]

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for SPY and AGG
# YOUR CODE HERE!
df_tech_stock_data = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

# Reorganize the DataFrame
# Separate ticker data
FB = df_tech_stock_data[df_tech_stock_data['symbol']=='FB'].drop('symbol', axis=1)
APPL = df_tech_stock_data[df_tech_stock_data['symbol']=='APPL'].drop('symbol', axis=1)
SPY = df_tech_stock_data[df_tech_stock_data['symbol']=='SPY'].drop('symbol', axis=1)
AMZN = df_tech_stock_data[df_tech_stock_data['symbol']=='AMZN'].drop('symbol', axis=1)
NFLX = df_tech_stock_data[df_tech_stock_data['symbol']=='NFLX'].drop('symbol', axis=1)
GOOG = df_tech_stock_data[df_tech_stock_data['symbol']=='GOOG'].drop('symbol', axis=1)


# Concatenate the ticker DataFrames
df_tech_stock_data = pd.concat([GOOG,APPL,SPY,AMZN,NFLX,FB],axis=1, keys=['GOOG','APPL','SPY','AMZN','NFLX','FB'])

# Preview DataFrame
df_tech_stock_data.head()
df_tech_stock_data.tail()

GOOG                                      \
                              open     high      low   close    volume   
timestamp                                                                
2022-08-31 04:00:00+00:00  111.630  111.770  109.050  109.15  25884888   
2022-09-01 04:00:00+00:00  109.200  111.220  108.190  110.55  22783719   
2022-09-02 04:00:00+00:00  111.340  111.675  108.130  108.68  20681559   
2022-09-06 04:00:00+00:00  108.135  108.880  106.510  107.48  20557394   
2022-09-07 04:00:00+00:00  107.760  110.990  107.615  110.48  23168179   

                                                  APPL           ...     NFLX  \
                          trade_count        vwap open high low  ...   volume   
timestamp                                                        ...            
2022-08-31 04:00:00+00:00      225813  110.003290  NaN  NaN NaN  ...  8615748   
2022-09-01 04:00:00+00:00      278560  109.577315  NaN  NaN NaN  ...  7292904   
2022-09-02 04:00:00+00:00      229808  109.749947  NaN  NaN NaN  ...  8163451   
2022-09-06 04:00:00+00:00      246791  107.564535  NaN  NaN NaN  ...  7465692   
2022-09-07 04:00:00+00:00      248533  109.595898  NaN  NaN NaN  ...  8147848   

                                                    FB                        \
                          trade_count        vwap open high low close volume   
timestamp                                                                      
2022-08-31 04:00:00+00:00      126819  227.669035  NaN  NaN NaN   NaN    NaN   
2022-09-01 04:00:00+00:00      123511  224.822034  NaN  NaN NaN   NaN    NaN   
2022-09-02 04:00:00+00:00      128928  229.017513  NaN  NaN NaN   NaN    NaN   
2022-09-06 04:00:00+00:00      129774  218.485956  NaN  NaN NaN   NaN    NaN   
2022-09-07 04:00:00+00:00      115048  224.690090  NaN  NaN NaN   NaN    NaN   

                                            
                          trade_count vwap  
timestamp                                   
2022-08-31 04:00:00+00:00         NaN  NaN  
2022-09-01 04:00:00+00:00         NaN  NaN  
2022-09-02 04:00:00+00:00         NaN  NaN  
2022-09-06 04:00:00+00:00         NaN  NaN  
2022-09-07 04:00:00+00:00         NaN  NaN  

[5 rows x 42 columns]

## ETFs
```
Vanguard S&P 500 ETF (VOO)
Vanguard Information Technology ETF (VGT)
```

In [20]:
# Format current date as ISO format
start_date = pd.Timestamp("2020-08-07", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-08-07", tz="America/New_York").isoformat()
# Set the tickers
tickers = ["VOO", "VGT"]

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for SPY and AGG
# YOUR CODE HERE!
df_etf = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

# Reorganize the DataFrame
# Separate ticker data
VOO = df_etf[df_etf['symbol']=='VOO'].drop('symbol', axis=1)
VGT = df_etf[df_etf['symbol']=='VGT'].drop('symbol', axis=1)

# Concatenate the ticker DataFrames
df_etf = pd.concat([VOO,VGT],axis=1, keys=['VOO','VGT'])

# Preview DataFrame
df_etf.head()
df_etf.tail()

VOO                                    \
                             open     high     low   close   volume   
timestamp                                                             
2022-08-01 04:00:00+00:00  376.12  380.010  375.44  377.65  4374908   
2022-08-02 04:00:00+00:00  376.11  379.627  373.95  375.21  4054107   
2022-08-03 04:00:00+00:00  377.19  382.100  376.91  381.07  3826838   
2022-08-04 04:00:00+00:00  380.88  381.530  379.19  380.77  3274628   
2022-08-05 04:00:00+00:00  376.60  380.700  376.54  379.98  2843831   

                                                      VGT                   \
                          trade_count        vwap    open    high      low   
timestamp                                                                    
2022-08-01 04:00:00+00:00       88166  377.840171  367.20  373.30  365.640   
2022-08-02 04:00:00+00:00       93254  376.597738  366.11  371.93  364.530   
2022-08-03 04:00:00+00:00       83236  380.034817  369.53  379.07  369.530   
2022-08-04 04:00:00+00:00       74004  380.565442  377.74  379.53  374.941   
2022-08-05 04:00:00+00:00       82991  378.877859  373.35  380.12  373.060   

                                                                   
                            close  volume trade_count        vwap  
timestamp                                                          
2022-08-01 04:00:00+00:00  369.54  572617       23880  370.083886  
2022-08-02 04:00:00+00:00  367.63  547046       26805  368.244635  
2022-08-03 04:00:00+00:00  378.29  482865       14945  376.632144  
2022-08-04 04:00:00+00:00  379.49  324378       11706  377.862618  
2022-08-05 04:00:00+00:00  378.90  338058       14358  377.190347

### Analyze the Performance

#### Tech Stocks

In [27]:
# extract close price form tech stock data 
tech_close_price = pd.DataFrame()
for i in [ticker[0] for ticker in df_tech_stock_data]:
    tech_close_price[i] = df_tech_stock_data[i]['close']
print(tech_close_price)

                              GOOG  APPL     SPY     AMZN    NFLX      FB
timestamp                                                                
2020-08-07 04:00:00+00:00  1494.49   NaN  334.57  3167.46  494.73  268.44
2020-08-10 04:00:00+00:00  1496.10   NaN  335.56  3148.16  483.38  263.00
2020-08-11 04:00:00+00:00  1480.32   NaN  332.82  3080.67  466.93  256.13
2020-08-12 04:00:00+00:00  1506.62   NaN  337.31  3162.24  475.47  259.89
2020-08-13 04:00:00+00:00  1518.45   NaN  336.81  3161.02  481.33  261.30
...                            ...   ...     ...      ...     ...     ...
2022-08-31 04:00:00+00:00   109.15   NaN  395.18   126.77  223.56     NaN
2022-09-01 04:00:00+00:00   110.55   NaN  396.42   127.82  230.04     NaN
2022-09-02 04:00:00+00:00   108.68   NaN  392.24   127.51  226.11     NaN
2022-09-06 04:00:00+00:00   107.48   NaN  390.76   126.11  218.39     NaN
2022-09-07 04:00:00+00:00   110.48   NaN  397.78   129.48  228.96     NaN

[525 rows x 6 columns]


In [29]:
# Clean the tech data
close_price_adj = tech_close_price.dropna()
close_price_adj.head()

,GOOG,APPL,SPY,AMZN,NFLX,FB
timestamp,,,,,,
